In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('selected_col_v2.csv')
data = data.drop(['Unnamed: 0'], axis=1)
data = data.dropna()

In [19]:
# 50 features from the most 50th important features of the RF model
data_2 = data[['Rank',
 'Fruity_flavor',
 'COE_score',
 'geisha',
 'Altitude',
 'Citric_acidity',
 'Sweet_flavor',
 'Floral_flavor',
 'NuttyCocoa_flavor',
 'Year_2021',
 'Country_Nicaragua',
 'Tartaric_acidity',
 'Farm_santa rosa',
 'Year_2020',
 'GreenVegetative_flavor',
 'Complex_acidity',
 'Process_honey',
 'Winey_flavor',
 'Spices_flavor',
 'Malic_acidity',
 'Country_Guatemala',
 'Process_anaerobic',
 'Country_Costa Rica',
 'Farm_el cerro',
 'typica',
 'Lactic_acidity',
 'North_America',
 'Farm_el paraiso',
 'Long_aftertaste',
 'Process_natural',
 'Country_Ecuador',
 'Year_2016',
 'pacamara',
 'Asia',
 'Farm_san luis',
 'Creamy_body',
 'Country_Ethiopia',
 'caturra',
 'Process_washed',
 'Year_2022',
 'Clean_and_clear',
 'Country_Brazil',
 'Roasted_flavor',
 'Farm_platanares',
 'Europe',
 'Country_Burundi',
 'Country_México',
 'Country_Honduras',
 'Year_2017',
 'Country_El Salvador',
 'High_bid']]

In [5]:
# split training data and test data

train, test = train_test_split(data_2, test_size=0.2, random_state=42)

# split X y
def split_x_y(df):
    x = df.drop(['High_bid'], axis=1)
    y = df['High_bid']
    return x, y

x_train, y_train = split_x_y(train)
x_test, y_test = split_x_y(test)

# standardize
doscaling = 1
if (doscaling == 1):
    xscaler = preprocessing.StandardScaler().fit(x_train[['COE_score', 'Altitude']])
    # standardize feature values
    X_train_conti = xscaler.transform(x_train[['COE_score', 'Altitude']])
    X_train = np.concatenate((X_train_conti, x_train.drop(['COE_score', 'Altitude'], axis=1)), axis=1)
    X_test_conti = xscaler.transform(x_test[['COE_score', 'Altitude']])
    X_test = np.concatenate((X_test_conti, x_test.drop(['COE_score', 'Altitude'], axis=1)), axis=1)
else:
    X_train = x_train
    X_test = x_test


y_mean = y_train.mean()
Y_train_keep = y_train.copy()
Y_test_keep = y_test.copy()

Y_train = y_train - y_mean
Y_test = y_test - y_mean


# validation is the last 10% of training, subtraining is the first 90% of training
nvalid = int(X_train.shape[0] * 0.1)
nsubtrain = X_train.shape[0] - nvalid

X_subtrain = X_train[0:nsubtrain, :].astype('float32')
X_valid = X_train[nsubtrain:, :].astype('float32')
Y_subtrain = Y_train[0:nsubtrain].astype('float32')
Y_valid = Y_train[nsubtrain:].astype('float32')

Y_subtrain_keep = Y_train_keep[0:nsubtrain].astype('float32')
Y_valid_keep = Y_train_keep[nsubtrain:].astype('float32')

print("X_train shape = ", X_train.shape)
print("X_subtrain shape = ", X_subtrain.shape)
print("X_valid shape = ", X_valid.shape)
print("Y_subtrain shape = ", Y_subtrain.shape)
print("Y_valid shape = ", Y_valid.shape)
print("X_test shape = ", X_test.shape)

X_train shape =  (1652, 90)
X_subtrain shape =  (1487, 90)
X_valid shape =  (165, 90)
Y_subtrain shape =  (1487,)
Y_valid shape =  (165,)
X_test shape =  (414, 90)


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

In [8]:
X_train_t = torch.FloatTensor(X_train)
Y_train_t = torch.FloatTensor(Y_train).view(-1, 1)

X_subtrain_t = torch.FloatTensor(X_subtrain)
Y_subtrain_t = torch.FloatTensor(Y_subtrain).view(-1, 1)

X_valid_t = torch.FloatTensor(X_valid)
Y_valid_t = torch.FloatTensor(np.array(Y_valid)).view(-1, 1)

X_test_t = torch.FloatTensor(X_test)
Y_test_t = torch.FloatTensor(np.array(Y_test)).view(-1, 1)

In [21]:
batch_size = 100
epochs = 500
nminibatches = int(np.ceil(X_subtrain.shape[0]/batch_size))

In [30]:
# training step function

def output(model, optimizer, loss_f, eval_criterion):
    
    best_rmse = float('inf')
    best_step_count = 0
    
    training_rmse_history = []
    val_rmse_history = []
    
    for epoch_i in range(epochs):

        model.train()

        # shuffle the subtraining data for each epoch
        shuffle_indices = np.random.permutation(len(X_subtrain_t))
        X_subtrain_t_i = X_subtrain_t[shuffle_indices]
        Y_subtrain_t_i = Y_subtrain_t[shuffle_indices]

        # training step
        for step in range(0, X_subtrain_t.shape[0], batch_size):

            # minibatch
            X_minibatch_t = X_subtrain_t_i[step:step+batch_size]
            Y_minibatch_t = Y_subtrain_t_i[step:step+batch_size]

            # forward pass
            Y_minibatch_pred_t = model(X_minibatch_t)

            # compute loss
            loss = loss_f(Y_minibatch_pred_t, Y_minibatch_t)

            # zero out the gradients
            optimizer.zero_grad()

            # backward pass
            loss.backward()

            # update the weights
            optimizer.step()

            # evaluate the model on the training and validation set every 10 minibatches
            if step % (batch_size*100) == 0:

                model.eval()
                # training set
                Y_subtrain_pred_t = model(X_subtrain_t)
                training_rmse = torch.sqrt(eval_criterion(Y_subtrain_pred_t, Y_subtrain_t))
                training_rmse_history.append(training_rmse.item())

                # validation set
                Y_valid_pred_t = model(X_valid_t)
                val_rmse = torch.sqrt(eval_criterion(Y_valid_pred_t, Y_valid_t))
                val_rmse_history.append(val_rmse.item())

                # set back to training mode
                model.train()

                # check if the best model based on val_rmse
                if (val_rmse < best_rmse):
                    best_rmse = val_rmse
                    best_step_count = (epoch_i+1)*nminibatches + step/batch_size
                    no_improvement_count = 0

                    # save the best model
                    best_model = model

                else:
                    no_improvement_count += 1

                # if no improvement for 500 minibatches after best_step_count (another 5 val_rmses), stop training
                if (no_improvement_count >= 50):
                    print(f'Early stopping at epoch {epoch_i+1}, batch {int(step/batch_size)}.') 
                    print(f'Best Validation RMSE: {best_rmse:.4f} as best step {int(best_step_count)}.')
                    print('')
                    break

        if (no_improvement_count >= 50):
                    break

    # the RMSE on the test set of the best model
    best_model.eval()
    Y_test_pred_t = best_model(X_test_t)
    true_list = Y_test_t
    pred_list = Y_test_pred_t
    test_rmse = torch.sqrt(eval_criterion(Y_test_pred_t, Y_test_t))
    print(f'Test RMSE: {test_rmse:.4f}')
    
    return best_model, best_rmse, best_step_count, test_rmse, training_rmse_history, val_rmse_history, true_list, pred_list 

In [10]:
# define the MLP model with 4 hidden layers and dropout
class MLPWithDropout(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        super(MLPWithDropout, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.layer2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)
        self.layer3 = nn.Linear(hidden_size, hidden_size)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.5)
        self.layer4 = nn.Linear(hidden_size, hidden_size)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(0.5)
        self.output_layer = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = self.dropout1(self.relu1(self.layer1(x)))
        x = self.dropout2(self.relu2(self.layer2(x)))
        x = self.dropout3(self.relu3(self.layer3(x)))
        x = self.dropout4(self.relu4(self.layer4(x)))
        x = self.output_layer(x)
        return x

In [23]:
test_rmse_list = []
learning_rate_list = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1]
H_list = [20, 50, 100, 150, 200]

for h in H_list:

    for l in learning_rate_list:

        np.random.seed(1321)
        model_i = MLPWithDropout(input_size=X_subtrain.shape[1], hidden_size=h)
        sse = nn.MSELoss(reduction='sum')
        eval_criterion = nn.MSELoss()
        optimizer_i = optim.Adam(model_i.parameters(), lr=l)

        # training
        best_model_i, best_rmse_i, best_step_count_i, test_rmse_i, training_rmse_history_i, val_rmse_history_i, true_list_i, pred_list_i = output(model_i, optimizer_i, sse, eval_criterion)
        print(f'MLP(4 Hidden Layers & Dropout) with Learning Rate = {l}, H = {h}')
        print('----------------------------------------')
        test_rmse_list.append(test_rmse_i.item())

Test RMSE: 20.3646
MLP(4 Hidden Layers & Dropout) with Learning Rate = 0.0001, H = 20
----------------------------------------
Early stopping at epoch 301, batch 0.
Best Validation RMSE: 10.4087 as best step 3765.

Test RMSE: 16.2023
MLP(4 Hidden Layers & Dropout) with Learning Rate = 0.0005, H = 20
----------------------------------------
Early stopping at epoch 212, batch 0.
Best Validation RMSE: 8.7939 as best step 2430.

Test RMSE: 12.9232
MLP(4 Hidden Layers & Dropout) with Learning Rate = 0.001, H = 20
----------------------------------------
Early stopping at epoch 99, batch 0.
Best Validation RMSE: 8.9065 as best step 735.

Test RMSE: 14.2577
MLP(4 Hidden Layers & Dropout) with Learning Rate = 0.005, H = 20
----------------------------------------
Early stopping at epoch 73, batch 0.
Best Validation RMSE: 11.2768 as best step 345.

Test RMSE: 16.3383
MLP(4 Hidden Layers & Dropout) with Learning Rate = 0.01, H = 20
----------------------------------------
Early stopping at epoch

Early stopping at epoch 74, batch 0.
Best Validation RMSE: 7.2493 as best step 360.

Test RMSE: 17.8371
MLP(4 Hidden Layers & Dropout) with Learning Rate = 0.01, H = 200
----------------------------------------
Early stopping at epoch 55, batch 0.
Best Validation RMSE: 12.2823 as best step 75.

Test RMSE: 24.1819
MLP(4 Hidden Layers & Dropout) with Learning Rate = 0.05, H = 200
----------------------------------------
Early stopping at epoch 57, batch 0.
Best Validation RMSE: 19.2832 as best step 105.

Test RMSE: 24.7150
MLP(4 Hidden Layers & Dropout) with Learning Rate = 0.1, H = 200
----------------------------------------
Early stopping at epoch 138, batch 0.
Best Validation RMSE: 19.6465 as best step 1320.

Test RMSE: 25.1178
MLP(4 Hidden Layers & Dropout) with Learning Rate = 0.5, H = 200
----------------------------------------
Early stopping at epoch 104, batch 0.
Best Validation RMSE: 19.6613 as best step 810.

Test RMSE: 29.9616
MLP(4 Hidden Layers & Dropout) with Learning Rat